In [1]:
import os

In [2]:
%pwd

'/home/vibhav911/Documents/skills/Data Science/DS_Projects/Chicken-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/vibhav911/Documents/skills/Data Science/DS_Projects/Chicken-Disease-Classification'

In [5]:
# Entity Program
# This program is the written type of the files and directories
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from ChickenDisease.constants import *
from ChickenDisease.utils.common import read_yaml, create_directories

In [10]:
# Config Manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self):
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config
        

In [8]:
import os
import urllib.request as request
import zipfile
from ChickenDisease.utils.common import get_size
from ChickenDisease import logger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with following headers \n {headers}')
        else:
            logger.info(f'{filename} already exist with the size : {get_size(Path(self.config.local_data_file))}')
            
            
    def extract_zipfile(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e